In [1]:
import nzpy
import pandas as pd
import os
import numpy as np
import lnwtrue
import datetime

In [2]:
year = 2022
OUTPUT_DIR = 'output/output_smes'
kpi_name = 'Revenue : Corporate Solution - SME (S)'
micro_org_target_template = '../../resources/Micro-org KPI Setting Template 20210505.xlsx'

In [3]:
conn = nzpy.connect(user="CORP_CENT", password="password",host='10.50.78.21', port=5480, database="TDADEVBAE", securityLevel=1,logLevel=0)

# mooc = lnwtrue.mooc.MoocArea(conn, dummy=True)
mooc = lnwtrue.mooc.MoocArea(conn)
trueconnect = lnwtrue.trueconnect
score = lnwtrue.score

In [4]:
query_str = f"""
SELECT TM_KEY_MTH, PRODUCT, ORGID_H, TDS_PROVINCE
     , ORGID_AA
     , CUST_CATG
     , NEW_EXISTING
     , SUM(SUBS) AS SUBS
     , SUM(REV) AS REV
     , SUM(REV_ESTM) AS REV_ESTM
FROM
(
    SELECT C.TM_KEY_MTH, C.PRODUCT
        , MA.ORGID_H 
        , MA.TDS_PROVINCE
        , MA.ORGID_AA
        , CUST_CATG
        , CASE WHEN SUBSTR(C.ACTV_MTH_FLAG,1,4) = SUBSTR(C.TM_KEY_MTH,1,4) THEN 'NEW' ELSE 'EXISTING' END AS NEW_EXISTING
        , CASE WHEN C.PRODUCT = 'TVS' THEN
            CASE WHEN COALESCE(C.PRICEPLAN,'NA') <> '512' THEN SUM(C.ACTV_SUBS) END  --> Exclude Subs TrueID TV 
            ELSE SUM(C.ACTV_SUBS) 
            END AS SUBS 
        , SUM(C.AMT) AS REV
        , SUM(C.AMT_ESTM) AS REV_ESTM
    FROM CORPNAPPO.MV_EDM_REVENUE_MTH C
    LEFT OUTER JOIN CORPNAPPO.DIM_MOOC_AREA MA 
                ON MA.SCAB_CODE = C.SCAB_CODE
    WHERE C.PRODUCT IN ('PREPAY','POSTPAY','BROADBAND','TVS') 
    AND C.CUST_CATG = 'BUS_CORP_S'
    AND COALESCE(C.REV_TYPE,'NA') NOT IN ('INVO_MGNT', 'CASHBACK', 'REACTIVE','NAD')
    AND C.TM_KEY_MTH >= {year}01 --> Month for select
    AND C.TM_KEY_MTH <= {year}12 --> Month for select
    GROUP BY C.TM_KEY_MTH, C.PRODUCT
        , MA.ORGID_H 
        , MA.TDS_PROVINCE, MA.ORGID_AA
        , CUST_CATG
        , CASE WHEN C.PRODUCT_GROUP = 'INTER_SIM' THEN C.PRODUCT_GROUP ELSE 'LOCAL' END
        , CASE WHEN SUBSTR(C.ACTV_MTH_FLAG,1,4) = SUBSTR(C.TM_KEY_MTH,1,4) THEN 'NEW' ELSE 'EXISTING' END
        , COALESCE(C.PRICEPLAN,'NA')
) RM
GROUP BY TM_KEY_MTH, PRODUCT, ORGID_H, TDS_PROVINCE, ORGID_AA, CUST_CATG
        , NEW_EXISTING
ORDER BY 1,2,3,4
;
"""

data = pd.read_sql(query_str, conn)
# data.columns = [col.decode("utf-8") for col in data.columns]

In [5]:
data.to_excel('sme_sizes.xlsx', index=False)

In [6]:
data

,TM_KEY_MTH,PRODUCT,ORGID_H,TDS_PROVINCE,ORGID_AA,CUST_CATG,NEW_EXISTING,SUBS,REV,REV_ESTM
0,202201,BROADBAND,None,None,None,BUS_CORP_S,NEW,17.0,1817.35,1817.35
1,202201,BROADBAND,None,None,None,BUS_CORP_S,EXISTING,26.0,12037.07,12037.07
2,202201,BROADBAND,001,B094-BKK,100201A,BUS_CORP_S,EXISTING,30.0,16193.50,16193.50
3,202201,BROADBAND,001,B094-BKK,100102A,BUS_CORP_S,NEW,1.0,0.00,0.00
4,202201,BROADBAND,001,B094-BKK,100101A,BUS_CORP_S,NEW,1.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
27855,202211,TVS,94Z,Pattani,940401A,BUS_CORP_S,EXISTING,0.0,0.00,0.00
27856,202211,TVS,94Z,Pattani,940101A,BUS_CORP_S,EXISTING,NaN,0.00,0.00
27857,202211,TVS,95Z,Yala,950101A,BUS_CORP_S,EXISTING,NaN,0.00,0.00
27858,202211,TVS,95Z,Yala,950201A,BUS_CORP_S,EXISTING,1.0,0.00,0.00


In [7]:
def _aa_month_scores(row):
    month_scores = row.groupby('TM_KEY_MTH').sum()['REV']
    for m in range(1,13):
        tm_month = int(f'2021{m:02}')
        if tm_month not in month_scores.index:
            month_scores[tm_month] = 0
    score_aa = score.MonthlyScore(month_scores)
    return score_aa.weekly_scores

# def _aa_month_scores(row):
#     month_scores = row.groupby('TM_KEY_MTH').sum()['REV']
#     for m in range(1,13):
#         tm_month = int(f'2021{m:02}')
#         if tm_month not in month_scores.index:
#             month_scores[tm_month] = 0
#     score_aa = score.MonthlyScore(month_scores)
#     return score_aa

In [8]:
weekly_scores = data.groupby('ORGID_AA').apply(_aa_month_scores).reset_index()

IllegalMonthError: bad month number 13; must be 1-12

In [ ]:
weekly_scores.columns = ['ORGID_AA', 'WEEKLY_SCORES']

In [ ]:
aa_to_h = {}
for i,row in data.iterrows():
    aa_to_h[row['ORGID_AA']] = row['ORGID_H']

h_to_r, h_to_g = mooc.h_to_all()

In [ ]:
# aa_to_h, aa_to_r, aa_to_g = mooc.aa_to_all()

In [ ]:
weekly_scores['ORGID_H'] = weekly_scores['ORGID_AA'].map(aa_to_h)
weekly_scores['ORGID_R'] = weekly_scores['ORGID_H'].map(h_to_r)
weekly_scores['ORGID_G'] = weekly_scores['ORGID_H'].map(h_to_g)
# weekly_scores['ORGID_P'] = 'P'
weekly_scores['ORGID_P'] = weekly_scores['ORGID_G'].apply(lambda x: 'P' if x==x else x)

filter 530 aas

In [ ]:
# mo_tem = pd.read_excel(micro_org_target_template)
# aa_530 = mo_tem[mo_tem['Geo_Level']==5]['Geo_Code'].unique()

# weekly_scores = weekly_scores[weekly_scores['ORGID_AA'].isin(aa_530)]

In [ ]:
matrix = []
for row in weekly_scores['WEEKLY_SCORES']:
    matrix.append(row)

matrix = np.array(matrix)

In [ ]:
current_week = datetime.datetime.today().isocalendar()[1]
# tm_weeks = [f'20210{week_no:02}' for week_no in range(1, current_week+2)]
tm_weeks = [f'20210{week_no:02}' for week_no in range(1, 53)]

In [ ]:
for i, tm_week in enumerate(tm_weeks):
    weekly_scores[tm_week] = matrix[:, i]

In [ ]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

template

In [ ]:
lv_cols = ['ORGID_AA', 'ORGID_H', 'ORGID_R', 'ORGID_G', 'ORGID_P']

In [ ]:
template_gen = trueconnect.KPITemplateGenerator(kpi_name)

In [ ]:
for week_no in tm_weeks:
    vals = pd.Series()
    for lv_col in lv_cols:
        val = weekly_scores.groupby(lv_col).sum()[week_no]
        vals = vals.append(val)

    vals['PC'] = 'NO'

    template = template_gen.gen_weekly_microorg(vals, year, week_no)
    template.clean_no_id()
    template.save(f'{OUTPUT_DIR}/{kpi_name.replace(":", "")}-{week_no}.xlsx')

In [ ]:
trueconnect.MicroorgActualTemplate.combine_output(OUTPUT_DIR, 'combine/sme_s_all.xlsx')